[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation) 
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from operator import itemgetter


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_tokens=512, 
)

## 6. trim 설정
trimmer = trim_messages(
    max_tokens=65,       ## 최대 토큰 수 제한
    strategy="last",     ## 최근 메시지부터 시작해서 자르기
    token_counter=llm,   ## 모델 기준으로 토큰 수 계산 
    include_system=True, ## 시스템 메시지 포함
    allow_partial=False, ## 메시지 자르기 제한
    start_on="human",    ## human 메시지부터 자르기 시작
)

## 7. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
    )

## 8. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 9. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
while True:
    query = input('육아휴직과 관련된 질문하세요 >> ')

    if query.upper()== 'S':
        break

    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )

    print('\n' + '=' * 50 + '\n')

In [5]:
store

{'abc': InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직의 기간은 1년 이내로 규정되어 있습니다. 다만, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다:\n1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우\n2. 「한부모가족지원법」 제4조제1호의 부 또는 모\n3. 고용노동부령으로 정하는 장애아동의 부 또는 모\n\n이 정보는 남녀고용평등법 제19조 제2항에 명시되어 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-41da2b84-52a8-4f52-b536-e1bc0dd9d57a-0'), HumanMessage(content='육아휴직 조건은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위해 신청할 수 있으며, 이 경우 사업주는 이를 허용해야 합니다. 다만, 대통령령으로 정하는 경우에는 예외가 있을 수 있습니다. 육아휴직의 기간은 1년 이내로 설정되나, 부모가 모두 각각 3개월 이상 사용한 경우나 한부모 가족, 장애 아동의 부모인 경우에는 6개월 이내에서 추가로 사용할 수 있습니다. (제19조)', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-

In [6]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]:{message.content}\n')
    

[HUMAN]:육아휴직 기간은 어떻게 되나요?

[AI]:육아휴직의 기간은 1년 이내로 규정되어 있습니다. 다만, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다:
1. 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우
2. 「한부모가족지원법」 제4조제1호의 부 또는 모
3. 고용노동부령으로 정하는 장애아동의 부 또는 모

이 정보는 남녀고용평등법 제19조 제2항에 명시되어 있습니다.

[HUMAN]:육아휴직 조건은 어떻게 되나요?

[AI]:육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위해 신청할 수 있으며, 이 경우 사업주는 이를 허용해야 합니다. 다만, 대통령령으로 정하는 경우에는 예외가 있을 수 있습니다. 육아휴직의 기간은 1년 이내로 설정되나, 부모가 모두 각각 3개월 이상 사용한 경우나 한부모 가족, 장애 아동의 부모인 경우에는 6개월 이내에서 추가로 사용할 수 있습니다. (제19조)

[HUMAN]:육아휴직 대상자는 어떻게 되나요?

[AI]:육아휴직의 대상자는 임신 중인 여성 근로자와 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하는 근로자입니다. 이 경우의 자녀는 입양한 자녀도 포함됩니다. 이는 "남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률" 제19조 제1항에 명시되어 있습니다.

[HUMAN]:육아휴직 수당은 얼마인가요?

[AI]:남녀고용평등법 자체에는 육아휴직 수당의 구체적인 금액에 대한 내용이 포함되어 있지 않습니다. 다만, 제18조에서는 출산전후휴가 등의 급여에 대해서 '통상임금에 상당하는 금액'을 지급할 수 있다고 언급하고 있습니다. 육아휴직 수당의 구체적인 금액은 보통 고용보험법에 따라 정해지며, 일반적으로 육아휴직 중인 근로자는 육아휴직 기간 동안 월평균 보수의 일정 비율을 수당으로 받게 됩니다.

구체적인 금액은 정부의 지원 정책에 따라 다를 수 있으므로, 고용노동부나 고용보험의 관련 규정을